In [3]:
from main import *

# define a class for friends activity analysis
class FriendActivityAnaliser:
    # define the constructor method that takes the database name as an input
    def __init__(self, database_name="friends_activity.db"):
        # store the database name as an attribute
        self.database_name = database_name
        # try to connect to the database and create a cursor object
        try:
            self.conn = sqlite3.connect(self.database_name)
            self.cur = self.conn.cursor()
            print(f"Connected to {self.database_name} successfully.")
        except sqlite3.Error as e:
            print(f"Error connecting to {self.database_name}: {e}")

    # define a method that returns the most played tracks for a given user or users, time period, artist or artists, album or albums and limit
    def top_tracks(self, user_id=None, time_period=None, by_artist_uri=None, by_album_uri=None, limit=None):
        # construct the SQL query to select the user name, user uri, track uri, track name, artist uri, album uri, album name and count the number of streamings for each track
        sql = """SELECT u.user_name, u.user_url, u.user_image_url, t.track_uri, t.track_name, al.album_name, al.album_uri, a.artist_uri, a.artist_name, COUNT(s.track_id) AS streamings
                FROM users u
                JOIN streamings s ON u.user_id = s.user_id -- join the users and streamings tables on user_id
                JOIN tracks t ON s.track_id = t.track_id -- join the streamings and tracks tables on track_id
                JOIN artists a ON t.artist_id = a.artist_id -- join the tracks and artists tables on artist_id
                JOIN albums al ON t.album_id = al.album_id -- join the tracks and albums tables on album_id
                """

        # initialize a list to store the query parameters
        params = []

        # if user_id is not None, add a WHERE clause to filter by user_id
        if user_id is not None:
            # if user_id is a single value, use the equal operator
            if isinstance(user_id, int):
                sql += " WHERE u.user_id = ?"
                params.append(user_id)
            # if user_id is a list or tuple of values, use the IN operator
            elif isinstance(user_id, (list, tuple)):
                sql += f" WHERE u.user_id IN ({','.join(['?'] * len(user_id))})"
                params.extend(user_id)

        # if time_period is not None, add a AND clause to filter by timestamp
        if time_period is not None:
            # if time_period is a tuple of start and end values, use the BETWEEN operator
            if isinstance(time_period, tuple) and len(time_period) == 2:
                sql += " AND s.timestamp BETWEEN ? AND ?"
                params.extend(time_period)
            # if time_period is a single value for start or end, use the greater than or equal or less than or equal operator
            elif isinstance(time_period, str):
                # check if the value starts with '>' or '<' and use the corresponding operator
                if time_period.startswith('>'):
                    sql += " AND s.timestamp >= ?"
                    params.append(time_period[1:])
                elif time_period.startswith('<'):
                    sql += " AND s.timestamp <= ?"
                    params.append(time_period[1:])

        # if by_artist_uri is not None, add a AND clause to filter by artist_uri
        if by_artist_uri is not None:
            # if by_artist_uri is a single value, use the equal operator
            if isinstance(by_artist_uri, str):
                sql += " AND a.artist_uri = ?"
                params.append(by_artist_uri)
            # if by_artist_uri is a list or tuple of values, use the IN operator
            elif isinstance(by_artist_uri, (list, tuple)):
                sql += f" AND a.artist_uri IN ({','.join(['?'] * len(by_artist_uri))})"
                params.extend(by_artist_uri)

        # if by_album_uri is not None, add a AND clause to filter by album_uri
        if by_album_uri is not None:
            # if by_album_uri is a single value, use the equal operator
            if isinstance(by_album_uri, str):
                sql += " AND al.album_uri = ?"
                params.append(by_album_uri)
            # if by_album_uri is a list or tuple of values, use the IN operator
            elif isinstance(by_album_uri, (list, tuple)):
                sql += f" AND al.album_uri IN ({','.join(['?'] * len(by_album_uri))})"
                params.extend(by_album_uri)

        # group by user name, track uri, track name, artist uri, album uri and album name
        sql += " GROUP BY u.user_name, t.track_uri, t.track_name, a.artist_uri , al.album_uri , al.album_name"

        # order by streamings in descending order
        sql += " ORDER BY streamings DESC"

        # if limit is not None and is an integer value greater than zero, add a LIMIT clause to limit the result to the given limit value
        if limit is not None and isinstance(limit, int) and limit > 0:
            sql += f" LIMIT {limit}"

        # try to execute the query and fetch the results
        try:
            self.cur.execute(sql, params)
            data = self.cur.fetchall()       
            # assume the above list is stored in a variable called data
            result = [] # initialize an empty list to store the result
            # loop through the data list
            for item in data:
                # create a dict for each item with the corresponding keys and values
                user_dict = {
                    'userName': item[0],
                    'userUri': item[1],
                    'userImage': item[2],
                    # create a dict for the track information
                    'track_info': {
                        'trackUri': item[3],
                        'trackName': item[4],
                        'albumUri': item[6],
                        'albumName': item[5],
                        'artistUri': item[7],
                        'artistName': item[8],
                        'streamings': item[9]
                    }
                }
                # check if the user name is already in the result list
                found = False # initialize a flag to indicate if the user is found
                for user in result:
                    # if the user name matches, append the track info to the existing user's tracks_info list
                    if user['userName'] == user_dict['userName']:
                        user['tracks_info'].append(user_dict['track_info'])
                        found = True # set the flag to True
                        break # exit the loop
                # if the user name is not found, create a new user dict with an empty tracks_info list and append the track info to it
                if not found:
                    user_dict['tracks_info'] = [user_dict.pop('track_info')] # remove the track_info key and add it as a list to tracks_info key
                    result.append(user_dict) # append the user dict to the result list

            # return the results as a list of tuples
            return result
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")


    # define a method that returns the most played artists for a given user or users, time period, and limit
    def top_artists(self, user_id=None, time_period=None, limit=None):
        # construct the SQL query to select the user name, artist uri, artist name, and count the number of streamings for each artist
        sql = """SELECT u.user_name, a.artist_uri, a.artist_name, COUNT(s.track_id) AS streamings
                FROM users u
                JOIN streamings s ON u.user_id = s.user_id -- join the users and streamings tables on user_id
                JOIN tracks t ON s.track_id = t.track_id -- join the streamings and tracks tables on track_id
                JOIN artists a ON t.artist_id = a.artist_id -- join the tracks and artists tables on artist_id
                """

        # initialize a list to store the query parameters
        params = []

        # if user_id is not None, add a WHERE clause to filter by user_id
        if user_id is not None:
            # if user_id is a single value, use the equal operator
            if isinstance(user_id, int):
                sql += " WHERE u.user_id = ?"
                params.append(user_id)
            # if user_id is a list or tuple of values, use the IN operator
            elif isinstance(user_id, (list, tuple)):
                sql += f" WHERE u.user_id IN ({','.join(['?'] * len(user_id))})"
                params.extend(user_id)

        # if time_period is not None, add an AND clause to filter by timestamp
        if time_period is not None:
            # if time_period is a tuple of start and end values, use the BETWEEN operator
            if isinstance(time_period, tuple) and len(time_period) == 2:
                sql += " AND s.timestamp BETWEEN ? AND ?"
                params.extend(time_period)
            # if time_period is a single value for start or end, use the greater than or equal or less than or equal operator
            elif isinstance(time_period, str):
                # check if the value starts with '>' or '<' and use the corresponding operator
                if time_period.startswith('>'):
                    sql += " AND s.timestamp >= ?"
                    params.append(time_period[1:])
                elif time_period.startswith('<'):
                    sql += " AND s.timestamp <= ?"
                    params.append(time_period[1:])

        # group by user name, artist uri, and artist name
        sql += " GROUP BY u.user_name, a.artist_uri, a.artist_name"

        # order by streamings in descending order
        sql += " ORDER BY streamings DESC"

        # if limit is not None and is an integer value greater than zero, add a LIMIT clause to limit the result to the given limit value
        if limit is not None and isinstance(limit, int) and limit > 0:
            sql += f" LIMIT {limit}"

        # try to execute the query and fetch the results
        try:
            self.cur.execute(sql, params)
            results = self.cur.fetchall()
            # return the results as a list of tuples
            return results
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")

    
    # define a function that returns the users that listened to a particular artist the most
    def top_users_by_artist(self, artist_id, limit=None):
        # construct the SQL query that joins the users, streamings, tracks, and artists tables on their respective columns and filters by the given artist_id
        sql = """SELECT u.user_name, u.user_image_url, COUNT(s.track_id) AS streamings
                FROM users u
                JOIN streamings s ON u.user_id = s.user_id
                JOIN tracks t ON s.track_id = t.track_id
                JOIN artists a ON t.artist_id = a.artist_id
                WHERE a.artist_id = ?
                GROUP BY u.user_name, u.user_image_url
                ORDER BY streamings DESC
            """

        # initialize a list to store the query parameters
        params = []

        # add the artist_id parameter to the list
        params.append(artist_id)

        # if limit is not None and is an integer value greater than zero, add a LIMIT clause to limit the result to the given limit value
        if limit is not None and isinstance(limit, int) and limit > 0:
            sql += f" LIMIT {limit}"

        # try to execute the query and fetch the results
        try:
            self.cur.execute(sql, params)
            results = self.cur.fetchall()
            # return the results as a list of tuples
            return results
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")
    # define a method that returns the most played albums for a given user or users, time period, and limit
    def top_albums(self, user_id=None, time_period=None, limit=None):
        # construct the SQL query to select the user name, album uri, album name, and count the number of streamings for each album
        sql = """SELECT u.user_name, al.album_uri, al.album_name, COUNT(s.track_id) AS streamings
                FROM users u
                JOIN streamings s ON u.user_id = s.user_id -- join the users and streamings tables on user_id
                JOIN tracks t ON s.track_id = t.track_id -- join the streamings and tracks tables on track_id
                JOIN albums al ON t.album_id = al.album_id -- join the tracks and albums tables on album id
                """

        # initialize a list to store the query parameters
        params = []

        # if user_id is not None, add a WHERE clause to filter by user_id
        if user_id is not None:
            # if user_id is a single value, use the equal operator
            if isinstance(user_id, int):
                sql += " WHERE u.user_id = ?"
                params.append(user_id)
            # if user_id is a list or tuple of values, use the IN operator
            elif isinstance(user_id, (list, tuple)):
                sql += f" WHERE u.user_id IN ({','.join(['?'] * len(user_id))})"
                params.extend(user_id)

        # if time_period is not None, add an AND clause to filter by timestamp
        if time_period is not None:
            # if time_period is a tuple of start and end values, use the BETWEEN operator
            if isinstance(time_period, tuple) and len(time_period) == 2:
                sql += " AND s.timestamp BETWEEN ? AND ?"
                params.extend(time_period)
            # if time_period is a single value for start or end, use the greater than or equal or less than or equal operator
            elif isinstance(time_period, str):
                # check if the value starts with '>' or '<' and use the corresponding operator
                if time_period.startswith('>'):
                    sql += " AND s.timestamp >= ?"
                    params.append(time_period[1:])
                elif time_period.startswith('<'):
                    sql += " AND s.timestamp <= ?"
                    params.append(time_period[1:])

        # group by user name, album uri, and album name
        sql += " GROUP BY u.user_name, al.album_uri, al.album_name"

        # order by streamings in descending order
        sql += " ORDER BY streamings DESC"

        # if limit is not None and is an integer value greater than zero, add a LIMIT clause to limit the result to the given limit value
        if limit is not None and isinstance(limit, int) and limit > 0:
            sql += f" LIMIT {limit}"

        # try to execute the query and fetch the results
        try:
            self.cur.execute(sql, params)
            results = self.cur.fetchall()
            # return the results as a list of tuples
            return results
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")

    # define a method that returns the most played artists for a given user or users, time period, and limit
    def top_contexts(self, user_id=None, time_period=None, limit=None, most_played_songs_in_the_context=1):
        # construct the SQL query to select the user name, context URI, context name, count the number of streamings for each context, and the most played song or songs in the context
        sql = """SELECT u.user_name, c.context_uri, c.context_name, COUNT(s.track_id) AS streamings,
                (SELECT t.track_name || ' (' || COUNT(s2.track_id) || ')' -- concatenate the track name and the number of streamings for each track in the context
                FROM streamings s2
                JOIN tracks t ON s2.track_id = t.track_id -- join the streamings and tracks tables on track_id
                WHERE s2.context_id = s.context_id -- filter by the same context_id as in the outer query
                GROUP BY t.track_name, t.track_id -- group by track name and track id
                ORDER BY COUNT(s2.track_id) DESC -- order by streamings in descending order
                LIMIT ? -- limit by the most_played_songs_in_the_context parameter
                ) AS most_played_songs_in_the_context
                FROM users u
                JOIN streamings s ON u.user_id = s.user_id -- join the users and streamings tables on user_id
                JOIN context c ON s.context_id = c.context_id -- join the streamings and context tables on context_id
            """

        # initialize a list to store the query parameters
        params = []

        # add the most_played_songs_in_the_context parameter to the list
        params.append(most_played_songs_in_the_context)

        # if user_id is not None, add a WHERE clause to filter by user_id
        if user_id is not None:
            # if user_id is a single value, use the equal operator
            if isinstance(user_id, int):
                sql += " WHERE u.user_id = ?"
                params.append(user_id)
            # if user_id is a list or tuple of values, use the IN operator
            elif isinstance(user_id, (list, tuple)):
                sql += f" WHERE u.user_id IN ({','.join(['?'] * len(user_id))})"
                params.extend(user_id)

        # if time_period is not None, add an AND clause to filter by timestamp
        if time_period is not None:
            # if time_period is a tuple of start and end values, use the BETWEEN operator
            if isinstance(time_period, tuple) and len(time_period) == 2:
                sql += " AND s.timestamp BETWEEN ? AND ?"
                params.extend(time_period)
            # if time_period is a single value for start or end, use the greater than or equal or less than or equal operator
            elif isinstance(time_period, str):
                # check if the value starts with '>' or '<' and use the corresponding operator
                if time_period.startswith('>'):
                    sql += " AND s.timestamp >= ?"
                    params.append(time_period[1:])
                elif time_period.startswith('<'):
                    sql += " AND s.timestamp <= ?"
                    params.append(time_period[1:])

        # group by user name, context URI, and context name
        sql += " GROUP BY u.user_name, c.context_uri, c.context_name"

        # order by streamings in descending order
        sql += " ORDER BY streamings DESC"

        # if limit is not None and is an integer value greater than zero, add a LIMIT clause to limit the result to the given limit value
        if limit is not None and isinstance(limit, int) and limit > 0:
            sql += f" LIMIT {limit}"

        # try to execute the query and fetch the results
        try:
            self.cur.execute(sql, params)
            results = self.cur.fetchall()
            
            # return the results as a list of tuples
            return results
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")# define a class for friends activity analysis


# friend_activity = FriendActivityAnaliser()

# f = friend_activity.top_tracks(limit=100)
# print(f)



In [5]:
# Import sqlite3 module
import sqlite3

# Connect to the database
conn = sqlite3.connect("friends_activity.db")

# Create a cursor object
cursor = conn.cursor()

# Get the names of all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Loop through each table
for table in tables:
    # Print the table name
    print(f"Table: {table[0]}")
    
    # Get the names of all columns in the table
    cursor.execute(f"PRAGMA table_info({table[0]})")
    columns = cursor.fetchall()
    
    # Print the column names
    print("Columns:")
    for column in columns:
        print(f"- {column[1]}")
    
    # Get the first 10 rows of data from the table
    cursor.execute(f"SELECT * FROM {table[0]} LIMIT 4")
    data = cursor.fetchall()
    
    # Print the data
    print("Data:")
    for row in data:
        print(row)
    
    # Print a blank line for separation
    print()


Table: users

Columns:

- user_id

- user_url

- user_name

...

Data:

(1, 1, '1684069017858', None)

(2, 2, '1684074584415', None)

(3, 3, '1684261183291', None)

(4, 4, '1684281732362', None)